In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer

from env import get_url
import wrangle_zillow

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Data Wrangling

## Zillow Data

### Acquire

#### Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

* #### Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

* #### Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.

* #### Only include properties that include a latitude and longitude value.

In [2]:
query = '''
SELECT prop.*, 
       pred1.logerror, 
       pred1.transactiondate, 
       air.airconditioningdesc, 
       arch.architecturalstyledesc, 
       build.buildingclassdesc, 
       heat.heatingorsystemdesc, 
       landuse.propertylandusedesc, 
       story.storydesc, 
       construct.typeconstructiondesc 
FROM   properties_2017 prop 
       LEFT JOIN predictions_2017 pred1 USING (parcelid) 
       INNER JOIN (SELECT parcelid, 
                          Max(transactiondate) maxtransactiondate 
                   FROM   predictions_2017 
                   GROUP  BY parcelid) pred2 
               ON pred1.parcelid = pred2.parcelid 
                  AND pred1.transactiondate = pred2.maxtransactiondate 
       LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
       LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
       LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
       LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
       LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
       LEFT JOIN storytype story USING (storytypeid) 
       LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE  prop.latitude IS NOT NULL 
	   AND prop.longitude IS NOT NULL; 
'''

url = get_url('zillow')

zillow = pd.read_sql(query, url)
zillow

id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0      1727539  14297519                    NaN                       NaN   
1      1387261  17052889                    NaN                       NaN   
2        11677  14186244                    NaN                       NaN   
3      2288172  12177905                    NaN                       NaN   
4      1970746  10887214                    1.0                       NaN   
...        ...       ...                    ...                       ...   
77376   673515  11000655                    NaN                       NaN   
77377  2968375  17239384                    NaN                       NaN   
77378  1843709  12773139                    1.0                       NaN   
77379  1187175  12826780                    NaN                       NaN   
77380  2455685  13083743                    NaN                       NaN   

       basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0               NaN          3.5         4.0                  NaN   
1               NaN          1.0         2.0                  NaN   
2               NaN          2.0         3.0                  NaN   
3               NaN          3.0         4.0                  NaN   
4               NaN          3.0         3.0                  NaN   
...             ...          ...         ...                  ...   
77376           NaN          2.0         2.0                  NaN   
77377           NaN          2.0         4.0                  NaN   
77378           NaN          1.0         3.0                  NaN   
77379           NaN          2.0         3.0                  NaN   
77380           NaN          2.0         3.0                  NaN   

       buildingqualitytypeid  calculatedbathnbr  decktypeid  \
0                        NaN                3.5         NaN   
1                        NaN                1.0         NaN   
2                        NaN                2.0         NaN   
3                        8.0                3.0         NaN   
4                        8.0                3.0         NaN   
...                      ...                ...         ...   
77376                    6.0                2.0         NaN   
77377                    NaN                2.0         NaN   
77378                    4.0                1.0         NaN   
77379                    6.0                2.0         NaN   
77380                    4.0                2.0         NaN   

       finishedfloor1squarefeet  calculatedfinishedsquarefeet  \
0                           NaN                        3100.0   
1                        1465.0                        1465.0   
2                           NaN                        1243.0   
3                           NaN                        2376.0   
4                           NaN                        1312.0   
...                         ...                           ...   
77376                       NaN                        1286.0   
77377                    1612.0                        1612.0   
77378                       NaN                        1032.0   
77379                       NaN                        1762.0   
77380                       NaN                        1050.0   

       finishedsquarefeet12  finishedsquarefeet13  finishedsquarefeet15  \
0                    3100.0                   NaN                   NaN   
1                    1465.0                   NaN                   NaN   
2                    1243.0                   NaN                   NaN   
3                    2376.0                   NaN                   NaN   
4                    1312.0                   NaN                   NaN   
...                     ...                   ...                   ...   
77376                1286.0                   NaN                   NaN   
77377                1612.0                   NaN                   NaN   
77378                1032.0                  

#### Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [3]:
zillow.describe()

id      parcelid  airconditioningtypeid  \
count  7.738100e+04  7.738100e+04           24953.000000   
mean   1.495139e+06  1.300715e+07               1.813289   
std    8.609071e+05  3.481346e+06               2.967894   
min    3.490000e+02  1.071186e+07               1.000000   
25%    7.520700e+05  1.153830e+07               1.000000   
50%    1.497932e+06  1.253157e+07               1.000000   
75%    2.240535e+06  1.421183e+07               1.000000   
max    2.982274e+06  1.676893e+08              13.000000   

       architecturalstyletypeid  basementsqft   bathroomcnt    bedroomcnt  \
count                206.000000     50.000000  77381.000000  77381.000000   
mean                   7.388350    679.720000      2.299130      3.053489   
std                    2.734542    689.703546      0.996651      1.139096   
min                    2.000000     38.000000      0.000000      0.000000   
25%                    7.000000    273.000000      2.000000      2.000000   
50%                    7.000000    515.000000      2.000000      3.000000   
75%                    7.000000    796.500000      3.000000      4.000000   
max                   21.000000   3560.000000     18.000000     16.000000   

       buildingclasstypeid  buildingqualitytypeid  calculatedbathnbr  \
count            15.000000           49672.000000       76772.000000   
mean              3.933333               6.534587           2.316867   
std               0.258199               1.721953           0.979755   
min               3.000000               1.000000           1.000000   
25%               4.000000               6.000000           2.000000   
50%               4.000000               6.000000           2.000000   
75%               4.000000               8.000000           3.000000   
max               4.000000              12.000000          18.000000   

       decktypeid  finishedfloor1squarefeet  calculatedfinishedsquarefeet  \
count       614.0               6023.000000                  77185.000000   
mean         66.0               1366.512535                   1785.219939   
std           0.0                671.308125                    954.049286   
min          66.0                 44.000000                    128.000000   
25%          66.0                955.000000                   1182.000000   
50%          66.0               1257.000000                   1543.000000   
75%          66.0               1615.000000                   2113.000000   
max          66.0               6912.000000                  35640.000000   

       finishedsquarefeet12  finishedsquarefeet13  finishedsquarefeet15  \
count          73749.000000             41.000000           3009.000000   
mean            1760.522543           1389.853659           2357.791625   
std              934.024340            123.531688           1188.775210   
min              128.000000           1056.000000            598.000000   
25%             1172.000000           1344.000000           1625.000000   
50%             1523.000000           1440.000000           2094.000000   
75%             2076.000000           1440.000000           2838.000000   
max            21929.000000           1560.000000          35640.000000   

       finishedsquarefeet50  finishedsquarefeet6          fips  fireplacecnt  \
count           6023.000000           386.000000  77381.000000   8276.000000   
mean            1381.771542          2082.500000   6048.819232      1.191155   
std              726.468878          1240.382784     20.748199      0.491080   
min               44.000000           380.000000   6037.000000      1.000000   
25%              956.000000           993.750000   6037.000000      1.000000   
50%             1259.000000          1812.500000   6037.000000      1.000000   
75%             1621.500000          3053.500000   6059.000000      1.000000   
max            12467.000000          5598.000000   6111.000000      5.000000   

        fullbathcnt  garagecarcnt  garaget

In [4]:
zillow.describe(include='object')

propertycountylandusecode propertyzoningdesc taxdelinquencyflag  \
count                      77381              50331               2886   
unique                        75               1907                  1   
top                         0100               LAR1                  Y   
freq                       26714               6747               2886   

       transactiondate airconditioningdesc architecturalstyledesc  \
count            77381               24953                    206   
unique             265                   5                      5   
top         2017-06-30             Central           Contemporary   
freq              1189               23133                    172   

                                     buildingclassdesc heatingorsystemdesc  \
count                                               15               49440   
unique                                               2                  10   
top     Buildings having wood or wood and steel frames             Central   
freq                                                14               33550   

              propertylandusedesc storydesc typeconstructiondesc  
count                       77381        50                  222  
unique                         13         1                    4  
top     Single Family Residential  Basement                Frame  
freq                        52320        50                  219

In [5]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null float64
finishedsquarefeet13            41 non-null float64
finishedsquarefeet15            3009 non-null float64
finishedsquarefeet50          

In [6]:
zillow.dtypes

id                                int64
parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                  float64
heatingorsystemtypeid           float64


In [7]:
zillow.shape

(77381, 68)

In [8]:
zillow.hist(figsize=(25 ,20))
plt.tight_layout()

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The colNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().colspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:313: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  if not layout[ax.rowNum + 1, ax.colNum]:
/usr/local/anaconda3/lib/python3.7/site-package

#### Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [9]:
def measure_na(df):
    na_df = pd.DataFrame(df.isna().sum(), columns=['num_na_rows'])
    na_df['pct_na_rows'] = df.isna().sum()/len(df.index)
    return na_df

In [10]:
measure_na(zillow)

num_na_rows  pct_na_rows
id                                      0     0.000000
parcelid                                0     0.000000
airconditioningtypeid               52428     0.677531
architecturalstyletypeid            77175     0.997338
basementsqft                        77331     0.999354
bathroomcnt                             0     0.000000
bedroomcnt                              0     0.000000
buildingclasstypeid                 77366     0.999806
buildingqualitytypeid               27709     0.358085
calculatedbathnbr                     609     0.007870
decktypeid                          76767     0.992065
finishedfloor1squarefeet            71358     0.922164
calculatedfinishedsquarefeet          196     0.002533
finishedsquarefeet12                 3632     0.046937
finishedsquarefeet13                77340     0.999470
finishedsquarefeet15                74372     0.961114
finishedsquarefeet50                71358     0.922164
finishedsquarefeet6                 76995     0.995012
fips                                    0     0.000000
fireplacecnt                        69105     0.893049
fullbathcnt                           609     0.007870
garagecarcnt                        51907     0.670798
garagetotalsqft                     51907     0.670798
hashottuborspa                      75843     0.980124
heatingorsystemtypeid               27941     0.361083
latitude                                0     0.000000
longitude                               0     0.000000
lotsizesquarefeet                    8239     0.106473
poolcnt                             61244     0.791460
poolsizesum                         76514     0.988796
pooltypeid10                        76917     0.994004
pooltypeid2                         76307     0.986121
pooltypeid7                         62339     0.805611
propertycountylandusecode               0     0.000000
propertylandusetypeid                   0     0.000000
propertyzoningdesc                  27050     0.349569
rawcensustractandblock                  0     0.000000
regionidcity                         1471     0.019010
regionidcounty                          0     0.000000
regionidneighborhood                46491     0.600806
regionidzip                            50     0.000646
roomcnt                                 0     0.000000
storytypeid                         77331     0.999354
threequarterbathnbr                 67285     0.869529
typeconstructiontypeid              77159     0.997131
unitcnt                             26818     0.346571
yardbuildingsqft17                  74993     0.969140
yardbuildingsqft26                  77311     0.999095
yearbuilt                             263     0.003399
numberofstories                     59821     0.773071
fireplaceflag                       77209     0.997777
structuretaxvaluedollarcnt            112     0.001447
taxvaluedollarcnt                       1     0.000013
assessmentyear                          0     0.000000
landtaxvaluedollarcnt                   2     0.000026
taxamount                               5     0.000065
taxdelinquencyflag                  74495     0.962704
taxdelinquencyyear                  74495     0.962704
censustractandblock                   244     0.003153
logerror                                0     0.000000
transactiondate                         0     0.000000
airconditioningdesc                 52428     0.677531
architecturalstyledesc              77175     0.997338
buildingclassdesc                   77366     0.999806
heatingorsystemdesc                 27941     0.361083
propertylandusedesc                     0     0.000000
storydesc                           77331     0.999354
typeconstructiondesc                77159     0.997131

#### Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [11]:
def measure_na_rows(df):
    na_row_df = pd.DataFrame(df.isna().sum(axis=1).value_counts(sort=False), 
                      columns=['num_rows'])
    na_row_df = na_row_df.reset_index()
    na_row_df = na_row_df.rename(columns={'index': 'num_col_missing'})
    na_row_df['pct_cols_missing'] = na_row_df.num_col_missing / len(df.columns.tolist())
    return na_row_df

In [12]:
measure_na_rows(zillow)

num_col_missing  num_rows  pct_cols_missing
0                23         2          0.338235
1                24        13          0.352941
2                25        24          0.367647
3                26        65          0.382353
4                27       316          0.397059
5                28       455          0.411765
6                29      5270          0.426471
7                30      3455          0.441176
8                31      9891          0.455882
9                32     12579          0.470588
10               33     14782          0.485294
11               34     13327          0.500000
12               35      5148          0.514706
13               36      5775          0.529412
14               37      3620          0.544118
15               38      1925          0.558824
16               39       285          0.573529
17               40       230          0.588235
18               41        29          0.602941
19               42        23          0.617647
20               43        28          0.632353
21               44        78          0.647059
22               45        50          0.661765
23               46         5          0.676471
24               47         3          0.691176
25               48         3          0.705882

#### Now lets take both of those previous functions and add them to a wrangle file

In [13]:
zillow = wrangle_zillow.get_zillow_data()

In [14]:
wrangle_zillow.measure_na_columns(zillow)

num_na_rows  pct_na_rows
id                                      0     0.000000
parcelid                                0     0.000000
airconditioningtypeid               52428     0.677531
architecturalstyletypeid            77175     0.997338
basementsqft                        77331     0.999354
bathroomcnt                             0     0.000000
bedroomcnt                              0     0.000000
buildingclasstypeid                 77366     0.999806
buildingqualitytypeid               27709     0.358085
calculatedbathnbr                     609     0.007870
decktypeid                          76767     0.992065
finishedfloor1squarefeet            71358     0.922164
calculatedfinishedsquarefeet          196     0.002533
finishedsquarefeet12                 3632     0.046937
finishedsquarefeet13                77340     0.999470
finishedsquarefeet15                74372     0.961114
finishedsquarefeet50                71358     0.922164
finishedsquarefeet6                 76995     0.995012
fips                                    0     0.000000
fireplacecnt                        69105     0.893049
fullbathcnt                           609     0.007870
garagecarcnt                        51907     0.670798
garagetotalsqft                     51907     0.670798
hashottuborspa                      75843     0.980124
heatingorsystemtypeid               27941     0.361083
latitude                                0     0.000000
longitude                               0     0.000000
lotsizesquarefeet                    8239     0.106473
poolcnt                             61244     0.791460
poolsizesum                         76514     0.988796
pooltypeid10                        76917     0.994004
pooltypeid2                         76307     0.986121
pooltypeid7                         62339     0.805611
propertycountylandusecode               0     0.000000
propertylandusetypeid                   0     0.000000
propertyzoningdesc                  27050     0.349569
rawcensustractandblock                  0     0.000000
regionidcity                         1471     0.019010
regionidcounty                          0     0.000000
regionidneighborhood                46491     0.600806
regionidzip                            50     0.000646
roomcnt                                 0     0.000000
storytypeid                         77331     0.999354
threequarterbathnbr                 67285     0.869529
typeconstructiontypeid              77159     0.997131
unitcnt                             26818     0.346571
yardbuildingsqft17                  74993     0.969140
yardbuildingsqft26                  77311     0.999095
yearbuilt                             263     0.003399
numberofstories                     59821     0.773071
fireplaceflag                       77209     0.997777
structuretaxvaluedollarcnt            112     0.001447
taxvaluedollarcnt                       1     0.000013
assessmentyear                          0     0.000000
landtaxvaluedollarcnt                   2     0.000026
taxamount                               5     0.000065
taxdelinquencyflag                  74495     0.962704
taxdelinquencyyear                  74495     0.962704
censustractandblock                   244     0.003153
logerror                                0     0.000000
transactiondate                         0     0.000000
airconditioningdesc                 52428     0.677531
architecturalstyledesc              77175     0.997338
buildingclassdesc                   77366     0.999806
heatingorsystemdesc                 27941     0.361083
propertylandusedesc                     0     0.000000
storydesc                           77331     0.999354
typeconstructiondesc                77159     0.997131

In [15]:
wrangle_zillow.measure_na_rows(zillow)

num_col_missing  num_rows  pct_cols_missing
0                23         2          0.338235
1                24        13          0.352941
2                25        24          0.367647
3                26        65          0.382353
4                27       316          0.397059
5                28       455          0.411765
6                29      5270          0.426471
7                30      3455          0.441176
8                31      9891          0.455882
9                32     12579          0.470588
10               33     14782          0.485294
11               34     13327          0.500000
12               35      5148          0.514706
13               36      5775          0.529412
14               37      3620          0.544118
15               38      1925          0.558824
16               39       285          0.573529
17               40       230          0.588235
18               41        29          0.602941
19               42        23          0.617647
20               43        28          0.632353
21               44        78          0.647059
22               45        50          0.661765
23               46         5          0.676471
24               47         3          0.691176
25               48         3          0.705882

### Prepare

#### Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [16]:
zillow = zillow[zillow.propertylandusetypeid.isin([261, 262, 263, 264, 266, 268, 273, 276, 279])]
zillow

id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0      1727539  14297519                    NaN                       NaN   
1      1387261  17052889                    NaN                       NaN   
2        11677  14186244                    NaN                       NaN   
3      2288172  12177905                    NaN                       NaN   
4      1970746  10887214                    1.0                       NaN   
...        ...       ...                    ...                       ...   
77376   673515  11000655                    NaN                       NaN   
77377  2968375  17239384                    NaN                       NaN   
77378  1843709  12773139                    1.0                       NaN   
77379  1187175  12826780                    NaN                       NaN   
77380  2455685  13083743                    NaN                       NaN   

       basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0               NaN          3.5         4.0                  NaN   
1               NaN          1.0         2.0                  NaN   
2               NaN          2.0         3.0                  NaN   
3               NaN          3.0         4.0                  NaN   
4               NaN          3.0         3.0                  NaN   
...             ...          ...         ...                  ...   
77376           NaN          2.0         2.0                  NaN   
77377           NaN          2.0         4.0                  NaN   
77378           NaN          1.0         3.0                  NaN   
77379           NaN          2.0         3.0                  NaN   
77380           NaN          2.0         3.0                  NaN   

       buildingqualitytypeid  calculatedbathnbr  decktypeid  \
0                        NaN                3.5         NaN   
1                        NaN                1.0         NaN   
2                        NaN                2.0         NaN   
3                        8.0                3.0         NaN   
4                        8.0                3.0         NaN   
...                      ...                ...         ...   
77376                    6.0                2.0         NaN   
77377                    NaN                2.0         NaN   
77378                    4.0                1.0         NaN   
77379                    6.0                2.0         NaN   
77380                    4.0                2.0         NaN   

       finishedfloor1squarefeet  calculatedfinishedsquarefeet  \
0                           NaN                        3100.0   
1                        1465.0                        1465.0   
2                           NaN                        1243.0   
3                           NaN                        2376.0   
4                           NaN                        1312.0   
...                         ...                           ...   
77376                       NaN                        1286.0   
77377                    1612.0                        1612.0   
77378                       NaN                        1032.0   
77379                       NaN                        1762.0   
77380                       NaN                        1050.0   

       finishedsquarefeet12  finishedsquarefeet13  finishedsquarefeet15  \
0                    3100.0                   NaN                   NaN   
1                    1465.0                   NaN                   NaN   
2                    1243.0                   NaN                   NaN   
3                    2376.0                   NaN                   NaN   
4                    1312.0                   NaN                   NaN   
...                     ...                   ...                   ...   
77376                1286.0                   NaN                   NaN   
77377                1612.0                   NaN                   NaN   
77378                1032.0                  

#### Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row)

In [17]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

#### Decide how to handle the remaining missing values:

##### We will remove calculatedbathnbr because it is identical to bathroomcnt.

In [18]:
zillow = wrangle_zillow.get_zillow_data()
zillow

id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0      1727539  14297519                    NaN                       NaN   
1      1387261  17052889                    NaN                       NaN   
2        11677  14186244                    NaN                       NaN   
3      2288172  12177905                    NaN                       NaN   
4      1970746  10887214                    1.0                       NaN   
...        ...       ...                    ...                       ...   
77376   673515  11000655                    NaN                       NaN   
77377  2968375  17239384                    NaN                       NaN   
77378  1843709  12773139                    1.0                       NaN   
77379  1187175  12826780                    NaN                       NaN   
77380  2455685  13083743                    NaN                       NaN   

       basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0               NaN          3.5         4.0                  NaN   
1               NaN          1.0         2.0                  NaN   
2               NaN          2.0         3.0                  NaN   
3               NaN          3.0         4.0                  NaN   
4               NaN          3.0         3.0                  NaN   
...             ...          ...         ...                  ...   
77376           NaN          2.0         2.0                  NaN   
77377           NaN          2.0         4.0                  NaN   
77378           NaN          1.0         3.0                  NaN   
77379           NaN          2.0         3.0                  NaN   
77380           NaN          2.0         3.0                  NaN   

       buildingqualitytypeid  calculatedbathnbr  decktypeid  \
0                        NaN                3.5         NaN   
1                        NaN                1.0         NaN   
2                        NaN                2.0         NaN   
3                        8.0                3.0         NaN   
4                        8.0                3.0         NaN   
...                      ...                ...         ...   
77376                    6.0                2.0         NaN   
77377                    NaN                2.0         NaN   
77378                    4.0                1.0         NaN   
77379                    6.0                2.0         NaN   
77380                    4.0                2.0         NaN   

       finishedfloor1squarefeet  calculatedfinishedsquarefeet  \
0                           NaN                        3100.0   
1                        1465.0                        1465.0   
2                           NaN                        1243.0   
3                           NaN                        2376.0   
4                           NaN                        1312.0   
...                         ...                           ...   
77376                       NaN                        1286.0   
77377                    1612.0                        1612.0   
77378                       NaN                        1032.0   
77379                       NaN                        1762.0   
77380                       NaN                        1050.0   

       finishedsquarefeet12  finishedsquarefeet13  finishedsquarefeet15  \
0                    3100.0                   NaN                   NaN   
1                    1465.0                   NaN                   NaN   
2                    1243.0                   NaN                   NaN   
3                    2376.0                   NaN                   NaN   
4                    1312.0                   NaN                   NaN   
...                     ...                   ...                   ...   
77376                1286.0                   NaN                   NaN   
77377                1612.0                   NaN                   NaN   
77378                1032.0                  

In [19]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null float64
finishedsquarefeet13            41 non-null float64
finishedsquarefeet15            3009 non-null float64
finishedsquarefeet50          

#### Changes that were made to nulls

* `calculatedfinishedsquarefeet's` were filled with the median
* `calculatedbathnbr` was removed because it is identical to bathroomcnt
* `buildingqualitytypeid` were filled with the median
* `finishedsquarefeet12` was filled with the median
* `heatingorsystemtypeid` was removed because it is an id for `heatingorsystemdesc`
* `lotsizesquarefeet` was filled with median
* rows that were missing `propertyzoningdesc`, `regionidcity`, `regionidzip`, `censustractandblock`, and `heatingsystemdesc` were removed
* `regionidneighborhood` was removed because almost half of the values are missing.
* `structuretaxvaluedollarcnt` was filled with the median.
* `yearbuilt` and `taxamount` were filled with the median

In [20]:
zillow = wrangle_zillow.prep_zillow()
zillow.info()

/Users/ryanmccall/codeup-data-science/ds-methodologies-exercises/clustering/wrangle_zillow.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zillow.calculatedfinishedsquarefeet[zillow.calculatedfinishedsquarefeet.isna()] = zillow.calculatedfinishedsquarefeet.median()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 69734 entries, 0 to 77380
Data columns (total 31 columns):
id                              69734 non-null int64
parcelid                        69734 non-null int64
bathroomcnt                     69734 non-null float64
bedroomcnt                      69734 non-null float64
buildingqualitytypeid           69734 non-null float64
calculatedfinishedsquarefeet    69734 non-null float64
finishedsquarefeet12            69734 non-null float64
fips                            69734 non-null float64
latitude                        69734 non-null float64
longitude                       69734 non-null float64
lotsizesquarefeet               69734 non-null float64
propertycountylandusecode       69734 non-null object
propertylandusetypeid           69734 non-null float64
propertyzoningdesc              69734 non-null object
rawcensustractandblock          69734 non-null float64
regionidcity                    69734 non-null float64
regionidcounty   

In [21]:
zillow.fips.value_counts()

6037.0    44229
6059.0    19789
6111.0     5716
Name: fips, dtype: int64